In [1]:
import glob
import re

In [2]:
%load_ext rpy2.ipython

/home/timp/miniconda3/envs/cpowgs/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex


Ok - trying to use prokka - roary to make a pangenome

Made a conda environment conda create --name cpowgs
Then conda install -c conda-forge -c bioconda -c defaults prokka
then
conda config --add channels r\
conda config --add channels defaults\
conda config --add channels conda-forge\
conda config --add channels bioconda\
conda install roary\
then\
conda install jupyterlab (I know, wrong order)\
then\
conda install raxml\
then\
conda install snp-sites\
then\
conda install pandas\
then\
conda install rpy2\
then\
conda install r-tidyverse\

In [3]:
%cd /atium/Data/Nanopore/cpowgs/trees/200408_timp_tree

/atium/Data/Nanopore/cpowgs/trees/200408_timp_tree


ok - clean up the pilon assmeblies so that the headers don't have _pilon_ over and over and over

In [5]:
!sed -i 's/_pilon//g' esco/*fasta
!sed -i 's/_pilon//g' klpn/*fasta
!sed -i 's/_pilon//g' encl/*fasta

Ok - I need to pull down the genbank files for Enterobacter cloacae: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
E. Coli: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
and K. Pneumo: https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz

In [6]:
%mkdir refs
%cd refs
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff ecoli_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
!gunzip *gz
%mv *gbff encl_ref.gb
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/240/185/GCF_000240185.1_ASM24018v2/GCF_000240185.1_ASM24018v2_genomic.gbff.gz
!gunzip *gz
%mv *gbff kpn_ref.gb
%cd ..

/atium/Data/Nanopore/cpowgs/trees/200408_timp_tree/refs
--2020-04-08 14:03:56--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 2607:f220:41e:250::10
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3462090 (3.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.gbff.gz’

GCF_000005845.2_ASM 100%[===================>]   3.30M  11.2MB/s    in 0.3s    

2020-04-08 14:03:56 (11.2 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.gbff.gz’ saved [3462090/3462090]

--2020-04-08 14:03:57--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/025/565/GCF_000025565.1_ASM2556v1/GCF_000025565.1_ASM2556v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 2607:f220:41e:250::10
Connecting to ftp.ncbi.nlm.nih.gov (ft

Ok - now prokka

In [7]:
for spec in ['encl', 'esco', 'klpn']:
    !echo prok_{spec}

prok_encl
prok_esco
prok_klpn


In [8]:
for spec in ['encl', 'esco', 'klpn']:
    %rm -R prok_$spec
    %rm -R gff_$spec
    %mkdir gff_$spec
    filelist=(glob.glob(spec+"/*fasta"))
    for myfile in filelist:
        tag=re.split('[\/\.]', myfile)[1]
        print(tag)
        outy='prok_'+spec+'/'+tag
        ref='refs/'+spec+'_ref.gb'
        !prokka --quiet --protein $ref --outdir $outy --prefix $tag --locustag $tag $myfile 
        %cp {outy}/*gff gff_$spec

rm: cannot remove 'prok_encl': No such file or directory
rm: cannot remove 'gff_encl': No such file or directory
ENCL_73
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok_encl/ENCL_73/ENCL_73.gbf
[tbl2asn-forever] Correcting dates in prok_encl/ENCL_73/ENCL_73.sqn
[tbl2asn-forever] Dates changed from 01-JAN-2019 to 08-APR-2020
ENCL_80
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok_encl/ENCL_80/ENCL_80.gbf
[tbl2asn-forever] Correcting dates in prok_encl/ENCL_80/ENCL_80.sqn
[tbl2asn-forever] Dates changed from 01-JAN-2019 to 08-APR-2020
ENCL_149
[tbl2asn-forever] Found Prokka input, correcting dates in .gbf|.sqn files.
[tbl2asn-forever] Correcting dates in prok_encl/ENCL_149/ENCL_149.gbf
[tbl2asn-forever] Correcting dates in prok_encl/ENCL_149/ENCL_149.sqn
[tbl2asn-forever] Dates changed from 01-JAN-2019 to 08-APR-2020
ENCL_175
[tbl2asn-forever] Found 

Ok - now let's try roary

Roary says first https://github.com/microgenomics/tutorials/blob/master/pangenome.md
you gotta organize the gff files

In [30]:
#for spec in ['encl', 'esco', 'klpn']:
for spec in ['klpn']:
    !roary -i 95 -f pan_{spec} -e -n -g 100000 -p 11 gff_{spec}/*gff

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.
Use of uninitialized value in require at (eval 8511) line 1.


Counting snps from the multi-fa using snp-sites tool

In [31]:
for spec in ['encl', 'esco', 'klpn']:
    !snp-sites -v pan_{spec}/core_gene_alignment.aln >pan_{spec}/core_gene_alignment.vcf

In [32]:
%%R 
library(tidyverse)

for (spec in c("encl", "esco", "klpn")) {

    vcfs=read_tsv(paste0("pan_", spec, "/core_gene_alignment.vcf"), comment="##")

    ##ok - for each sample column, need to look at delta for all other sample columns, then sum to get that square on the table
    ##Interesting names are names(vcfs)[10:22]
    
    lastcol=dim(vcfs)[2]
    #print(lastcol)
    
    comp=as_data_frame(t(combn(names(vcfs[10:lastcol]),2))) %>%
        mutate(val=colSums(abs(vcfs[V1]-vcfs[V2])))

    comp=rbind(comp, data_frame(V1=comp$V2, V2=comp$V1, val=comp$val))

    write_csv(spread(comp, V2, val), paste0("pan_", spec, "/core_gene_alignment_count.csv"))
}

cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)


cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)


cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)




Ok - that generated a pan genome and core alignment file which is already getting me close.  I then ran it in geneious, but let me take a guess and try to run in raxml
some tips:
-p is a random parsimony seed - to make sure analysis does the same thing
-m GTRGAMMA - GTR GAMMA method
-x is the rapid bootstrapping seed
-N is the number of bootstrapops
-f a says do rapid bootstrap

In [33]:
#for spec in ['encl', 'esco', 'klpn']:
for spec in ['klpn']:
    %rm -R raxml_{spec}
    %mkdir raxml_{spec}
    %cd raxml_{spec}
    !raxmlHPC-PTHREADS -T 11 -f a -m GTRGAMMA -p 53243 -x 72341 -s ../pan_{spec}/core_gene_alignment.aln -N 100 -n testrapid.tree
    %cd ..

/atium/Data/Nanopore/cpowgs/trees/200408_timp_tree/raxml_klpn

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



IMPORTANT WARNING: Sequences KLPN_574 and KLPN_596 are exactly identical

IMPORTANT WARNING
Found 1 sequence that is exactly identical to other sequences in the alignment.
Normally they should be excluded from the analysis.

Just in case you might need it, an alignment file with 
sequence duplicates removed is printed to file ../pan_klpn/core_gene_alignment.aln.reduced


Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 2

This is RAxML Worker Pthread Number: 1

This is RAxML Worker Pthread Number: 3

This is RAxML Worker Pthread Number: 4

This is RAxML Worker Pthread Number: 9

This is RAxML Worker Pthread Number: 10

This is RAxML Worker Pthread Number: 8

This is RAxML Worker Pthread Number: 7

This is RAxML Worke

Ok - Trish wants to remove KLPN 637 and ENCL 86 from the trees because they are too divergent so:

In [ ]:
!cp -R pan_klpn pan_klpn_full
!cp -R gff_klpn gff_klpn_full
!rm gff_klpn/KLPN_637.gff


In [37]:
!cp -R pan_encl pan_encl_full
!cp -R gff_encl gff_encl_full
!rm gff_encl/ENCL_86.gff


In [38]:
#for spec in ['encl', 'esco', 'klpn']:
for spec in ['encl','klpn']:
    !roary -i 95 -f pan_{spec} -e -n -g 100000 -p 11 gff_{spec}/*gff

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.
Use of uninitialized value in require at (eval 12223) line 1.
Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Mar

In [40]:
for spec in ['encl', 'esco', 'klpn']:
    !snp-sites -v pan_{spec}/core_gene_alignment.aln >pan_{spec}/core_gene_alignment.vcf

In [41]:
%%R 
library(tidyverse)

for (spec in c("encl", "esco", "klpn")) {

    vcfs=read_tsv(paste0("pan_", spec, "/core_gene_alignment.vcf"), comment="##")

    ##ok - for each sample column, need to look at delta for all other sample columns, then sum to get that square on the table
    ##Interesting names are names(vcfs)[10:22]
    
    lastcol=dim(vcfs)[2]
    #print(lastcol)
    
    comp=as_data_frame(t(combn(names(vcfs[10:lastcol]),2))) %>%
        mutate(val=colSums(abs(vcfs[V1]-vcfs[V2])))

    comp=rbind(comp, data_frame(V1=comp$V2, V2=comp$V1, val=comp$val))

    write_csv(spread(comp, V2, val), paste0("pan_", spec, "/core_gene_alignment_count.csv"))
}

cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)


cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)


cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)




In [ ]:
#for spec in ['encl', 'esco', 'klpn']:
for spec in ['encl','klpn']:
    %rm -R raxml_{spec}
    %mkdir raxml_{spec}
    %cd raxml_{spec}
    !raxmlHPC-PTHREADS -T 11 -f a -m GTRGAMMA -p 53243 -x 72341 -s ../pan_{spec}/core_gene_alignment.aln -N 100 -n testrapid.tree
    %cd ..

Ooooooooook - now we think that ENCL_86 is KLPN and KLPN_637 is ENCL?

In [4]:
!cp -R pan_klpn pan_klpn_leftout
!cp -R gff_klpn gff_klpn_leftout
!cp -R pan_encl pan_encl_leftout
!cp -R gff_encl gff_encl_leftout


In [20]:
!cp gff_encl_full/ENCL_86.gff gff_klpn/ENCL_86.gff
!cp gff_klpn_full/KLPN_637.gff gff_encl/KLPN_637.gff

In [21]:
!rm gff_encl/ENCL_86.gff
!rm gff_klpn/KLPN_637.gff

rm: cannot remove 'gff_encl/ENCL_86.gff': No such file or directory
rm: cannot remove 'gff_klpn/KLPN_637.gff': No such file or directory


In [11]:
!rm -R pan_encl
!rm -R pan_klpn

In [22]:
#for spec in ['encl', 'esco', 'klpn']:
for spec in ['encl','klpn']:
    !roary -i 95 -f pan_{spec} -e -n -g 100000 -p 11 gff_{spec}/*gff

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Martin Hunt, Vanessa K. Wong, Sandra Reuter, Matthew T. G. Holden, Maria Fookes, Daniel Falush, Jacqueline A. Keane, Julian Parkhill,
	"Roary: Rapid large-scale prokaryote pan genome analysis", Bioinformatics, 2015 Nov 15;31(22):3691-3693
    doi: http://doi.org/10.1093/bioinformatics/btv421
	Pubmed: 26198102

Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.
Use of uninitialized value in require at (eval 12431) line 1.
Use of uninitialized value in require at /home/timp/miniconda3/envs/cpowgs/lib/site_perl/5.26.2/x86_64-linux-thread-multi/Encode.pm line 61.

Please cite Roary if you use any of the results it produces:
    Andrew J. Page, Carla A. Cummins, Mar

In [23]:
for spec in ['encl', 'esco', 'klpn']:
    !snp-sites -v pan_{spec}/core_gene_alignment.aln >pan_{spec}/core_gene_alignment.vcf

In [16]:
%%R 
library(tidyverse)

for (spec in c("encl", "esco", "klpn")) {

    vcfs=read_tsv(paste0("pan_", spec, "/core_gene_alignment.vcf"), comment="##")

    ##ok - for each sample column, need to look at delta for all other sample columns, then sum to get that square on the table
    ##Interesting names are names(vcfs)[10:22]
    
    lastcol=dim(vcfs)[2]
    #print(lastcol)
    
    comp=as_data_frame(t(combn(names(vcfs[10:lastcol]),2))) %>%
        mutate(val=colSums(abs(vcfs[V1]-vcfs[V2])))

    comp=rbind(comp, data_frame(V1=comp$V2, V2=comp$V1, val=comp$val))

    write_csv(spread(comp, V2, val), paste0("pan_", spec, "/core_gene_alignment_count.csv"))
}

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

R[write to console]: ✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_character(),
  INFO = col_character(),
  FORMAT = col_character()
)

R[write to console]: See spec(...) for full column specifications.

R[write to console]: Parsed with column specification:
cols(
  .default = col_double(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  QUAL = col_character(),
  FILTER = col_characte

In [ ]:
#for spec in ['encl', 'esco', 'klpn']:
for spec in ['encl','klpn']:
    %rm -R raxml_{spec}
    %mkdir raxml_{spec}
    %cd raxml_{spec}
    !raxmlHPC-PTHREADS -T 11 -f a -m GTRGAMMA -p 53243 -x 72341 -s ../pan_{spec}/core_gene_alignment.aln -N 100 -n testrapid.tree
    %cd ..

rm: cannot remove 'raxml_encl': No such file or directory
/atium/Data/Nanopore/cpowgs/trees/200408_timp_tree/raxml_encl

RAxML can't, parse the alignment file as phylip file 
it will now try to parse it as FASTA file



IMPORTANT WARNING: Sequences ENCLR_86 and ENCL_338 are exactly identical

IMPORTANT WARNING
Found 1 sequence that is exactly identical to other sequences in the alignment.
Normally they should be excluded from the analysis.

An alignment file with sequence duplicates removed has already
been printed to file ../pan_encl/core_gene_alignment.aln.reduced


Using BFGS method to optimize GTR rate parameters, to disable this specify "--no-bfgs" 


This is the RAxML Master Pthread

This is RAxML Worker Pthread Number: 3

This is RAxML Worker Pthread Number: 2

This is RAxML Worker Pthread Number: 4

This is RAxML Worker Pthread Number: 1

This is RAxML Worker Pthread Number: 5

This is RAxML Worker Pthread Number: 10

This is RAxML Worker Pthread Number: 9

This is RAxML Worker

06/21/20 - Trish has requested MLST - using torst's mlst github:
% conda install -c conda-forge -c bioconda -c defaults mlst


In [ ]:
!mlst encl/*fasta --quiet --threads 8 --scheme ecloacae >encl_mlst.tsv
!mlst esco/*fasta --quiet --threads 8 --scheme ecoli >esco_mlst.tsv
!mlst klpn/*fasta --quiet --threads 8 --scheme kpneumoniae >klpn_mlst.tsv
